In [1]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://172.23.56.33:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Silences"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

ServerSelectionTimeoutError: host.docker.internal:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 667508d666e2af7bfe29a5fd, topology_type: Unknown, servers: [<ServerDescription ('host.docker.internal', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('host.docker.internal:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [2]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Phrases"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(5617,
 {'_id': ObjectId('6674df97b213e264c76d3ddc'),
  'FileId': '7bff6b80-7332-4cdf-bf24-cedffb9234ef',
  'FileName': 'transtfe_esit TRANSTFE-PRI24-ESIT-1_0.mp4',
  'FolderName': 'Trabajo Fin de Máster',
  'Phrase': 'para poder realizar',
  'TotalCount': 2})

In [4]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Keywords"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(132830,
 {'_id': ObjectId('6674e616559693247e9a5f1d'),
  'FileId': '78ed433f-2b64-4d76-806c-6b2efb0ac16e',
  'FileName': '150477.mp4',
  'FolderName': 'Trabajo Fin de Máster',
  'Keyword': 'problema',
  'TotalCount': 17})

In [21]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Transcription"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(50,
 {'_id': ObjectId('66728593d002deaab01dd4b3'),
  'FileId': 'a473cf3a-de7f-466f-ba5a-6b2fe51c31b4',
  'FileName': '132379.mp4',
  'FolderName': 'Herramientas de Visualización',
  'Duration': 2187.72,
  'StartDatetime': datetime.datetime(2024, 6, 19, 2, 11, 31, 753000),
  'EndDatetime': datetime.datetime(2024, 6, 19, 2, 15, 31, 93000),
  'SizeMb': 4.4110107421875,
  'Transcription': [{'Id': 0,
    'TranscriptId': '72334268-6bbd-4c43-aeed-e00dc87286b1',
    'StartTime': 0.0,
    'EndTime': 8.78,
    'Text': ' Bueno, pues muchísimas gracias por asistir a esta primera sesión de la asignatura de'},
   {'Id': 1,
    'TranscriptId': '2ae0bdeb-b500-4950-a2f9-f4abc62a7e5a',
    'StartTime': 8.78,
    'EndTime': 16.080000000000002,
    'Text': ' herramientas de visualización del Master de Visualización, perdón, el Master de Visual Analytics.'},
   {'Id': 2,
    'TranscriptId': 'ba5c48e8-72dd-4177-aa11-230b77f10571',
    'StartTime': 16.080000000000002,
    'EndTime': 23.8,
    'Text': ' Como

In [3]:
import pandas as pd
from functions.chroma_utils import *  # Asegúrate de que la importación sea correcta
import multiprocessing

# Nombre de la base de datos y colección
collection_chroma = init_chroma('Transcription')

def process_transcription(i):
    """Procesa una transcripción individualmente."""
    try:
        list_transcriptions = i['Transcription']
        df_transcriptions = pd.DataFrame(list_transcriptions)
        list_ids = df_transcriptions['TranscriptId'].tolist()
        list_text = df_transcriptions['Text'].tolist()
        df_metadata = df_transcriptions.drop(["Text", "TranscriptId"], axis=1)
        df_metadata['FileId'] = i['FileId']
        df_metadata['FileName'] = i['FileName']
        df_metadata['FolderName'] = i['FolderName']
        dict_metadata = df_metadata.to_dict(orient='records')
        filter = {
            "$and": [
                {"FolderName": i['FolderName']},
                {"FileName": i['FileName']}
            ]
        }
        save_data_chroma(collection_chroma, list_ids, list_text, dict_metadata)
        print(f"Transcription from {i['FileName']} processed.")
    except Exception as e:
        print(f"Error processing {i['FileName']}: {e}")

# Procesar el primer registro
if data:  # Verificar que 'data' no esté vacío
    process_transcription(data[0])

# Paralelizar el resto en lotes de 5
def process_batch(batch):
    for item in batch:
        process_transcription(item)

if __name__ == '__main__':
    if len(data) > 1:  # Verificar si hay más de un registro para paralelizar
        pool = multiprocessing.Pool(processes=5)
        for i in range(1, len(data), 5):
            batch = data[i:i+5]
            pool.apply_async(process_batch, args=(batch,))

        pool.close()
        pool.join()

Total de elementos en la colección: 0
Transcription from 132379.mp4 processed.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Transcription from 165212.mp4 processed.
Transcription from 150869.mp4 processed.
Transcription from 152993.mp4 processed.
Transcription from 130988.mp4 processed.
Transcription from 141021.mp4 processed.
Transcription from 157707.mp4 processed.
Transcription from 156432.mp4 processed.
Transcription from 163935.mp4 processed.
Transcription from 144149.mp4 processed.
Transcription from 134523.mp4 processed.
Transcription from 145411.mp4 processed.
Transcription from 130986.mp4 processed.
Transcription from 150872.mp4 processed.
Transcription from 145434.mp4 processed.
Transcription from 167425.mp4 processed.
Transcription from 136696.mp4 processed.
Transcription from 168376.mp4 processed.
Transcription from 153570.mp4 processed.
Transcription from 135130.mp4 processed.
Transcription from 160314.mp4 processed.
Transcription from 162374.mp4 processed.
Transcription from 146845.mp4 processed.
Transcription from 148625.mp4 processed.
Transcription from 165194.mp4 processed.
Transcription fr

In [2]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Silences"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find({'FileId': '753f4c9e-004e-43a4-95d3-474e41597f08'})

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(1,
 {'_id': ObjectId('6673d26a3694d27b53a7de5d'),
  'FileId': '753f4c9e-004e-43a4-95d3-474e41597f08',
  'TotalTranscriptionTime': 4987.860000000001,
  'FileName': '89315.mp4',
  'FolderName': 'Visualización Interactiva de la Información',
  'Duration': 5152.8,
  'SilenceTime': 164.9399999999996})

In [6]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 Perfecto.
 Pues muchas gracias por asistir a una nueva sesión de
 herramientas de visualización.
 Estamos con el primer gran bloque que tiene que ver con
 herramientas de visualización programáticas basadas en
 aplicaciones web.
 En este caso con HTML, JavaScript y D3,
 que es el framework que estamos utilizando y que,
 como hemos comentado en una sesión,
 nació y estaba en la red de la web.
 Y ahora estamos en la red de la web.
 Como hemos comentado en una sesión,
 nació y estaba totalmente orientado a la representación
 gráfica de contenido, ya sea texto,
 gráficos como veremos, incluye animaciones,
 incluye interacción.
 Y lo que vamos a ver hoy, la sesión se va a dividir en dos
 partes.
 Por un lado, la presentación de la actividad,
 que tiene que ver con la actividad grupal que se hace con
 D3.
 Y luego continuaremos con más ejercicios,
 con más elementos que hicimos, o sea,
 sobre el ejercicio que hicimos la semana pasada.
 Les voy a compartir mi pantalla.
 ¿Están viendo

In [3]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 PERTOR Pero tú puedearing ciertos
 co했던 hab
icating la
 que es el frenguo que estamos utilizando y que como hemos comentado
 de una sesión, nació y está totalmente orientado al representación gráfica
 de contenido, ya se atexo gráficos como veremos, incluyen invaciones,
 incluyen interactión. Y lo que vamos a ver hoy o la sesión se va a dividir
 en dos partes, por lo lado la presentación de interactividad,
 ¿vale?, que tiene que ver con la actividad global de que se ateconte
 de tres y luego continuaremos con más ejerterios con más elementos que hicimos
 o se sobre el ejerterio que hicimos la sesión a la base, ¿vale?,
 te voy a compartir mi pantalla,
 también no vi pantalla,
 perfecto, porque había varias preguntas o bastante preguntas y siempre he dudas
 alrededor de la actividad, ¿no?, como lo primero,
 la actividad simplemente había, creo que es importante explicarlo, ¿no?,
 el objetivo es o supuesto utilizar de tres, pero como siempre digo las herramientas
 de visuete en s

In [4]:
# Realizar la consulta (obtiene todos los documentos)
filter = {
    "FileName":"139008.mp4"
    }
cursor = collection.find(filter)

# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    print(documento)

In [25]:
result = collection.delete_many(filter)
print(result.deleted_count, "documentos eliminados")

0 documentos eliminados


In [26]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Transcription"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

result = collection.delete_many({})
print(result.deleted_count, "documentos eliminados")

155 documentos eliminados


In [7]:
# El TranscriptId que estás buscando
transcript_id = '4f9b6965-97d7-4c2a-90f3-015e098f4a53'

# Consulta para encontrar la transcripción específica
resultado = collection.find_one(
    {'Transcription.TranscriptId': transcript_id},
    {
        'Duration':1,
        'FileName':1,
        'FolderName':1,
        'Transcription.$': 1 
    }  # Proyección para devolver solo la transcripción específica
)

# Imprimir el resultado
print(resultado)

{'_id': ObjectId('666f9b50da1ca2495a147728'), 'FileName': '86970.mp4', 'FolderName': 'Análisis e interpretación de datos', 'Duration': 600, 'Transcription': [{'Id': 0, 'TranscriptId': '4f9b6965-97d7-4c2a-90f3-015e098f4a53', 'StartTime': 60.0, 'EndTime': 74.46000000000001, 'Text': ' Bueno, pues podemos empezar. Soy Jara Mesquita, voy a hacer gusto profesor en la'}]}


In [2]:
from functions.chroma_utils import *
collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")
search_chroma(collection_chroma, ['sobre 7 hombres, lo las mujeres'], {'FileId': 'f6a40845-711e-4394-b19b-5cb68bb72017'}, 5)

Total de elementos en la colección: 26831
Total de elementos en la colección: 26831


{'ids': [[]],
 'distances': None,
 'metadatas': [[]],
 'embeddings': [[]],
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

In [2]:
from functions.chroma_utils import *  # Asegúrate de tener la función init_chroma() definida aquí

collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")

# Realizar la búsqueda y almacenar los resultados
results = search_chroma(collection_chroma, ['en que clase se habla de mongo?'], {}, 5)

# Imprimir los resultados
if results['ids']:  # Verificar si hay resultados
    for i, document in enumerate(results['documents']):
        print(f"Resultado {i+1}: {document}")

        # Imprimir metadatos si es necesario (ajusta las claves según tus metadatos)
        print("Metadatos:", results['metadatas'][i])
else:
    print("No se encontraron resultados.")


Total de elementos en la colección: 32301
Total de elementos en la colección: 32301
Resultado 1: [' Mongo esto lo ha pensado muy bien. A mí esta parte de Mongo me gusta mucho, pero es verdad que,', ' de MongoSH, MongoShell. Si MongoDAM es un ejecutable, lo puedo invocar desde el mismo cliente. Ni de', ' pues vamos a hablar un pelín poquito de esto. Los de Mongo no son tontos. La gente de Mongo es muy buena.', ' usando llaves para el particionado. Aquí en Mongo no, Mongo es más de sharding.', ' para hablar de disponibilidad, claro, yo tengo que pensar en cómo las aplicaciones se comunican con Mongo.']
Metadatos: [{'EndTime': 1780.76, 'FileId': '3d3b8a0d-6dcb-419c-be8b-b46ab8450ad7', 'FileName': '106153.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 260, 'StartTime': 1775.56}, {'EndTime': 1354.56, 'FileId': '3d3b8a0d-6dcb-419c-be8b-b46ab8450ad7', 'FileName': '106153.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 195, 'StartTime': 1344.76}, {'EndTime': 1508.68,

In [27]:
from functions.mongo_utils import *

collection_mongo = init_mongodb("Metadata", "Transcription")
# Realizar la consulta (obtiene todos los documentos)
filter = {
    "FileName":"106153.mp4"
    }
cursor = collection_mongo.find(filter)

# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    print(documento)

{'_id': ObjectId('6673460c8f86dea5118acd69'), 'FileId': '3d3b8a0d-6dcb-419c-be8b-b46ab8450ad7', 'FileName': '106153.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Duration': 3723.76, 'StartDatetime': datetime.datetime(2024, 6, 19, 15, 47, 6, 725000), 'EndDatetime': datetime.datetime(2024, 6, 19, 15, 56, 44, 468000), 'SizeMb': 5.9326171875, 'Transcription': [{'Id': 0, 'TranscriptId': 'e891aea3-1d3c-4bb3-94dd-75e5530f68eb', 'StartTime': 0.0, 'EndTime': 7.44, 'Text': ' Hola chicos, ¿qué tal? Bienvenidos a la semana número 8 de la asignatura. Ya estamos ahora'}, {'Id': 1, 'TranscriptId': '98ba1437-205f-481b-8024-3ace9dc4c5e8', 'StartTime': 7.44, 'EndTime': 12.44, 'Text': ' sí en el ecuador de todo esto. Vale, hoy vamos a hacer un repaso, la clase de hoy la vamos'}, {'Id': 2, 'TranscriptId': 'aa2d1538-287e-46ff-9deb-fa88a4535e40', 'StartTime': 12.44, 'EndTime': 17.0, 'Text': ' a dedicar a hacer un repaso, un alto en el camino de todo lo que hemos visto sobre MongoDB,'}, {'Id': 3, '

: 

In [24]:
from functions.chroma_utils import *  # Asegúrate de tener la función init_chroma() definida aquí
from functions.mongo_utils import *

collection_mongo = init_mongodb("Metadata", "Transcription")
collection_chroma = init_chroma('Transcription')

# Realizar la búsqueda y almacenar los resultados
user_query = 'en que clase se habla de redes convolucionales?'
results_chroma = search_chroma(collection_chroma, [user_query], {}, 10)

# Imprimir los resultados
window = 10
list_candidates = []
if results_chroma['ids']:  # Verificar si hay resultados
    for i, document in enumerate(results_chroma['documents'][0]):
        # Imprimir metadatos si es necesario (ajusta las claves según tus metadatos)
        print(results_chroma['ids'][0][i])
        id = results_chroma['ids'][0][i]
        score = results_chroma['distances'][0][i]
        metadata = results_chroma['metadatas'][0][i]
        start_time = metadata['StartTime'] - window
        end_time = metadata['EndTime'] + window
        file_id = metadata['FileId']
        file_name = metadata['FileName']
        folder_name = metadata['FolderName']

        pipeline = [
            {
                "$project": {
                    "FileId": 1,
                    "Transcription": 1
                }
            },
            { "$unwind": "$Transcription" },
            {
                "$match": {
                    "$and": [
                        { "Transcription.StartTime": { "$gte": start_time } },
                        { "Transcription.EndTime": { "$lte": end_time } },
                        { "FileId": file_id }
                    ]
                }
            },
            {
                "$group": {
                    "_id": "$FileId",
                    "Transcriptions": { "$push": "$Transcription" }
                }
            }
        ]

        # Ejecutar la consulta
        results_mongo = aggregate_mongodb(collection_mongo, pipeline)
        transcriptions = results_mongo[0]['Transcriptions']
        candidate = {
            "Score":score,
            "Subject":folder_name,
            "Video":file_name,
            "StartTimeSeconds":transcriptions[0]['StartTime'],
            "EndTimeSeconds":transcriptions[-1]['EndTime'],
            "Transcript": ' '.join([i['Text'].strip() for i in transcriptions])
        }
        list_candidates.append(candidate)

        print(metadata['FileId'], metadata['FileName'], metadata['StartTime'], metadata['EndTime'])
else:
    print("No se encontraron resultados.")

list_candidates_sorted = sorted(list_candidates, key=lambda x: x['Score'], reverse=True)

Total de elementos en la colección: 114658
193ee672-a489-4d0f-90bf-839d355d816f
c7b57995-ed69-4733-b743-153ab6018128 140933.mp4 1312.0 1317.0
a3ce0561-7ada-410c-9165-e57260fa2e7e
d464dda4-9fa5-4f4a-985c-35bfa33c6612 165194.mp4 2045.0 2048.0
8742a327-61c8-4870-92d7-c21c35306eaf
d464dda4-9fa5-4f4a-985c-35bfa33c6612 165194.mp4 1283.0 1288.0
d284004e-dbd6-4e35-8a5c-9b5cabb9325e
d464dda4-9fa5-4f4a-985c-35bfa33c6612 165194.mp4 2161.0 2164.0
58943bd1-0d6a-413e-a86c-a3e60e1162c7
c7b57995-ed69-4733-b743-153ab6018128 140933.mp4 606.0 611.0
cc78f5a5-1a8b-4c20-b966-8adbae57f0c9
d464dda4-9fa5-4f4a-985c-35bfa33c6612 165194.mp4 2114.0 2118.0
5d1f1933-e6af-463f-80d8-c4e73fb75bf9
d464dda4-9fa5-4f4a-985c-35bfa33c6612 165194.mp4 2408.0 2409.0
b1a71589-5db8-4c02-bdfb-0e1917dc772c
c7b57995-ed69-4733-b743-153ab6018128 140933.mp4 247.0 250.0
fc2b8cb2-a998-42df-b39d-fcba2ed406c8
25cbc299-26fd-4701-97b3-0df5d8b62ddc 157707.mp4 334.96000000000004 340.44
c209855a-2757-4dc9-998d-f54b4854512f
7b109a06-f097-47c7-95

In [23]:
list_candidates_sorted

[{'Score': 0.7915185689926147,
  'Subject': 'Ingeniería para el Procesado Masivo de Datos',
  'Video': '102774.mp4',
  'StartTimeSeconds': 2413.52,
  'EndTimeSeconds': 2437.52,
  'Transcript': 'Es cierto que todas las transformaciones que son white, por norma general, tienen como dos etapas, digamos. Una parte que se hace localmente en cada executor, y otra parte donde el resultado parcial que tenemos en cada executor hay que moverlo de un sitio a otro.'},
 {'Score': 0.7664271593093872,
  'Subject': 'Sistemas Cognitivos Artificiales',
  'Video': '157707.mp4',
  'StartTimeSeconds': 328.48,
  'EndTimeSeconds': 349.2,
  'Transcript': 'va tiene vueltas de más nos podemos empezar a encontrar un poquito de de overfit incluso no porque el modelo ha sido capaz de aprender todas las características en pocas épocas vale y luego saltará a la siente más cosas que hemos visto la capa flat pues lo que contaba en alguna clase no'},
 {'Score': 0.7525537610054016,
  'Subject': 'Sistemas Cognitivos Arti

In [28]:
import json
from functions.openai_utils import *
client_openai = init_openai()

In [29]:
messages = [{"role": "system", "content": "Eres un asistente que responde a las dudas del usuario e incluye los temas clave de cada posible respuesta, además de devolver la referencia según el mejor candidato elegido para responder las respuesta del usaurio."},
                {"role": "user", "content": user_query},
                {"role": "user", "content": f'##Candidates: {json.dumps(list_candidates_sorted)}'}]
response_gpt, usage = completion_openai(client_openai, 'gpt-3.5-turbo', messages)
response_gpt, usage

('En la clase de **Sistemas Cognitivos Artificiales** se habla de redes convolucionales. En estas sesiones se abordan temas como el desarrollo de modelos de aprendizaje profundo, el funcionamiento de las redes neuronales convolucionales, su aplicación en tareas de visión por computadora, entre otros temas relacionados con el procesamiento de datos y el aprendizaje automático. Te recomendaría revisar el video "Sistemas Cognitivos Artificiales" correspondiente a la referencia **157707.mp4** para obtener más información sobre las redes convolucionales mencionadas en clase.',
 CompletionUsage(completion_tokens=135, prompt_tokens=1520, total_tokens=1655))

In [33]:
prompt = f'''
Ayudas a determinar los mejores candidatos para responder a la petición del cliente.
### UserQuery: {user_query}
### Candidates: {json.dumps(list_candidates_sorted[0:5])}
'''
messages = [{"role": "user", "content": prompt}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_response",
            "description": "Respondes a las dudas del usuarios en base a los datos candidatos.",
            "parameters": {
                "type": "object",
                "properties": {
                    "UserResponse": {
                        "type": "string",
                        "description": "Contiene una respuesta coherente a la duda del usuario.", 
                        },
                    "References": {
                        "type": "string",
                        "description": "Contiene las referencias al mejor candidato para responder al usuario.", 
                        },
                    "Keywords": {
                        "type": "string",
                        "description": "Obtiene las palabras clave más relevantes en base a la temática." 
                        },
                    "SubjectName": {
                        "type": "string",
                        "description": "Nombre de la materia que es mejor candidato." 
                        },
                    "VideoName": {
                        "type": "string",
                        "description": "Nombre del video que es mejor candidato." 
                        }
                },
                "required": ['UserResponse', 'References', 'Keywords', 'SubjectName', 'VideoName'],
            },
        },
    }
]

tool, arguments, usage = tool_openai(client_openai,'gpt-3.5-turbo',messages,tools)
tool, arguments, usage

('get_response',
 {'UserResponse': 'En la clase de Ingeniería para el Procesado Masivo de Datos se habla de redes convolucionales.',
  'References': '102774.mp4',
  'Keywords': 'redes convolucionales, Ingeniería para el Procesado Masivo de Datos',
  'SubjectName': 'Ingeniería para el Procesado Masivo de Datos',
  'VideoName': '102774.mp4'},
 CompletionUsage(completion_tokens=91, prompt_tokens=892, total_tokens=983))